In [35]:
import os
import re
import calendar
import yaml
import numpy as np
import pandas as pd
from glob import glob
from datetime import datetime

all_codes = r'(^NM[CILPG][PX][0-9]+)'
project_codes = r'(^NM[ACIL]P[0-9]+)'

input_folder = r'finance_reports'
config = yaml.load(open('config.yaml', "r"), Loader=yaml.FullLoader)
excel_header_row = 5
project_pattern = re.compile(project_codes)
report_dateformat = '%B %Y'
PROFIT_AND_LOSS = 'profit and loss'


def get_currency(company_name):

    if 'companies' not in config.keys():
        raise ValueError('No companies definied in config') 

    for k, v in config['companies'].items():
        
        if 'name' not in v.keys():
            raise ValueError(f'Company has no name: "{k}"')
        if 'currency' not in v.keys():
            raise ValueError(f'Company has no currency: "{k}"')
        
        if v['name'].lower() == company_name.lower():
            return v['currency']

    raise ValueError(f'Could not find "{company_name}" in config')
    

def get_conversion_rate(company_name, year, month):

    if 'exchange_rates' not in config.keys():
        raise ValueError('No exchange_rates definied in config') 

    for v in config['exchange_rates']:
        
        if 'year' not in v.keys():
            raise ValueError(f'Year missing from exchange_rate item: "{v}"')
        if 'month' not in v.keys():
            raise ValueError(f'Month missing from exchange_rate item: "{v}"')
            
        if str(v['year']) == year:
            if str(v['month']).lower() == month:
                currency = get_currency(company_name)
                if currency not in v.keys():
                    raise ValueError(f'"{currency}" not found in excahnge_rate item: "{v}"')
                return v[currency], currency
                
    raise ValueError(f'Could not find conversion rate for "{company_name}" on "{date.strftime(report_dateformat)}" in config')


def get_mps_category(cost):

    if 'cost_mapping' not in config.keys():
        raise ValueError('No cost_mapping definied in config') 

    for category, v in config['cost_mapping'].items():
        if cost.lower() in [x.lower() for x in v]:
            return category

    raise ValueError(f'Found unmapped cost project cost: "{cost}" - add this mapping to the config.')


def is_p_and_l_report(path):

    # get the company name, report type and date - ignore if not a p&l
    df = pd.read_excel(path, header=None)
    report = str(df.iloc[1, 0]).lower()

    return report == PROFIT_AND_LOSS


def get_mps_project_categories(df):

    # check all the project cost columns are mapped
    try:
        start_project_costs = np.nonzero(df.columns=='projects')[0][0] + 1
    except IndexError:
        raise ValueError(f'Could not find "projects" in costs categories')
    try:
        end_project_costs = np.nonzero(df.columns=='total projects')[0][0]
    except IndexError:
        raise ValueError(f'Could not find "total projects" in costs categories')
    categories = [[cost, get_mps_category(cost)] for cost in df.columns[start_project_costs:end_project_costs]]

    return pd.DataFrame(categories, columns=['finance', 'mps'])


def get_mps_income_categories(df):

    # check all the project cost columns are mapped
    try:
        start_income = np.nonzero(df.columns=='income')[0][0] + 1
    except IndexError:
        raise IndexError(f'Could not find "income" in costs categories')
    try:
        end_income = np.nonzero(df.columns=='total income')[0][0]
    except IndexError:
        raise IndexError(f'Could not find "total income" in costs categories')
    categories = [[cost, get_mps_category(cost)] for cost in df.columns[start_income:end_income]]

    return pd.DataFrame(categories, columns=['finance', 'mps'])


# drop columns that are not project costs or income and merge into the mps categories
def get_mps_columns(df):

    mps_project_categories = get_mps_project_categories(df)  # check all the project cost columns are mapped
    try:
        mps_income_categories = get_mps_income_categories(df)  # check all the project cost columns are mapped
    except IndexError:
        mps_income_categories = pd.DataFrame([], columns=['finance', 'mps'])

    # merge the columns by the mps group
    finance_columns = pd.concat([mps_project_categories.finance, mps_income_categories.finance]).values
    mps_columns = pd.concat([mps_project_categories.mps, mps_income_categories.mps]).values
    df = df[finance_columns].T
    df['mps_code'] = mps_columns
    df = df.groupby('mps_code').sum().T

    return df


def get_date(val):

    if re.match(r'^[1-9]+-[0-9]+ [a-zA-Z]+, 20[2-9][0-9]$', val):  # 'day-day month, year'
        tmp = re.split(',| |-', val)
        date = datetime.strptime(f'{tmp[-3]} {tmp[-1]}', report_dateformat)
        day = tmp[1]   
        
    elif re.match(r'^[a-zA-Z]+ [1-9]+-[0-9]+, 20[2-9][0-9]$', val):   # 'month day-day, year'
        tmp = re.split(',| |-', val)
        date = datetime.strptime(f'{tmp[0]} {tmp[-1]}', report_dateformat)
        day = tmp[2] 
        
    elif re.match(r'^[a-zA-Z]+ 20[2-9][0-9]$', val):  # 'month year'
        date = datetime.strptime(val, report_dateformat)
        day = calendar.monthrange(date.year, date.month)[1]
        
    year = date.strftime('%Y')
    month = date.strftime('%B').lower()
        
    return year, month, day
                

def import_profit_and_loss_report(path):

    # get the company name, report type and date - ignore if not a p&l
    df = pd.read_excel(path, header=None)
    company_name = str(df.iloc[0, 0]).lower()
    report = str(df.iloc[1, 0]).lower()

    if report != PROFIT_AND_LOSS:
        raise ValueError(f'"{path}" is not the expected format for a profit and loss report')

    year, month, day = get_date(df.iloc[2, 0])

    # now just get the data
    df = pd.read_excel(path, header=excel_header_row-1, index_col=0)

    # get rid of columns that do not have a project code
    # ignore the final row and the empty rows before it 
    # final row is something like: 'Monday, Jul 10, 2023 01:44:15 pm GMT+1 - Accrual Basis'
    project_columns = [column for column in df.columns if project_pattern.match(column)]
    df = df[project_columns][:-1].T
    df.columns = [str(x).lower().lstrip() for x in df.columns]

    # drop columns that are not project costs or income and merge into the mps categories
    df = get_mps_columns(df)
    
    # parse out the project code and name and add them back in as seperate columns
    projec_codes_and_names = [project_pattern.split(project)[1:] for project in df.index]  
    tmp = pd.DataFrame(projec_codes_and_names, columns=['code', 'name'])  

    # get the exchange rate and convert to GBP
    rate, currency = get_conversion_rate(company_name, year, month)
    df[df.select_dtypes(include=['number']).columns] *= rate

    # add in the dataframe specific information to the columns
    df.insert(0, 'Project Name', tmp.name.values)
    df.insert(0, 'Project Code', tmp.code.values)
    df.insert(0, 'Conversion Rate', rate)
    df.insert(0, 'Converted From', currency)
    df.insert(0, 'Currency', 'GBP')
    df.insert(0, 'Year', year)
    df.insert(0, 'Month', month)
    df.insert(0, 'Day', day)
    df.insert(0, 'Report', report)
    df.insert(0, 'Company', company_name)

    print(f'Parsed {company_name} {report} {month} {year} using exchange rate {rate}')

    return df

def get_mps_report(folder):
    # parse all the xls files in a folder
    reports = []
    for path in glob(os.path.join(folder, '*.xlsx')):
        if is_p_and_l_report(path):
            try:
                reports.append(import_profit_and_loss_report(path))
            except Exception as e:
                print(f'Unexpected error with report: {path} - {e}')
                raise e
        else:
            print(f'WARNING: Ignoring "{path}", format does not match profit and loss report.')

    return pd.concat(reports).fillna(0)
    

df = get_mps_report(input_folder)
df.to_csv('result.csv')
display(df)

Parsed network mapping corp profit and loss july 2023 using exchange rate 0.58
Parsed network mapping corp profit and loss june 2023 using exchange rate 0.58
Parsed network mapping inc profit and loss july 2023 using exchange rate 0.76
Parsed network mapping inc profit and loss june 2023 using exchange rate 0.76
Parsed network mapping limited profit and loss july 2023 using exchange rate 1.0
Parsed network mapping limited profit and loss june 2023 using exchange rate 1.0
Parsed network mapping pty ltd profit and loss july 2023 using exchange rate 0.52
Parsed network mapping pty ltd profit and loss june 2023 using exchange rate 0.52


mps_code,Company,Report,Day,Month,Year,Currency,Converted From,Conversion Rate,Project Code,Project Name,...,Travel and Subsistence,IS,Salaries - Ops,Salaries - Project Management,Aviation,Income,Internal Trade,Shipping,Subcontractors,Salaries - Production
NMCP23017 HONI 2023,network mapping corp,profit and loss,10,july,2023,GBP,CAD,0.58,NMCP23017,HONI 2023,...,6167.7490,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00
NMCP22118 Altalink Bull trail,network mapping corp,profit and loss,30,june,2023,GBP,CAD,0.58,NMCP22118,Altalink Bull trail,...,1777.6710,0.0000,637.2808,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00
NMIP23020 FPL Tx Spring 2023,network mapping corp,profit and loss,30,june,2023,GBP,CAD,0.58,NMIP23020,FPL Tx Spring 2023,...,0.0000,0.0000,0.0000,256.1976,0.0000,0.0000,0.0000,0.0000,0.00,0.00
NMCP23017 HONI 2023,network mapping corp,profit and loss,30,june,2023,GBP,CAD,0.58,NMCP23017,HONI 2023,...,2797.9432,1045.8560,2867.7636,1537.1856,0.0000,0.0000,0.0000,0.0000,0.00,0.00
NMIP23042 NGUS TLE 2023,network mapping inc,profit and loss,10,july,2023,GBP,USD,0.76,NMIP23042,NGUS TLE 2023,...,499.0616,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00
NMIP23035 NYPA Group B,network mapping inc,profit and loss,10,july,2023,GBP,USD,0.76,NMIP23035,NYPA Group B,...,108.7104,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00,0.00
NMIP23019 GPC SoCo VM 2023,network mapping inc,profit and loss,10,july,2023,GBP,USD,0.76,NMIP23019,GPC SoCo VM 2023,...,0.0000,0.0000,0.0000,0.0000,53598.9392,0.0000,0.0000,0.0000,0.00,0.00
NMIP23056 APC SoCo VM 2023,network mapping inc,profit and loss,10,july,2023,GBP,USD,0.76,NMIP23056,APC SoCo VM 2023,...,0.0000,0.0000,0.0000,0.0000,20295.7316,0.0000,0.0000,0.0000,0.00,0.00
NMCP22118 ALtalink Bull Wind,network mapping inc,profit and loss,30,june,2023,GBP,USD,0.76,NMCP22118,ALtalink Bull Wind,...,0.0000,0.0000,0.0000,228.6384,0.0000,0.0000,0.0000,0.0000,0.00,0.00
NMIP23005 Ameren Spring 2023,network mapping inc,profit and loss,30,june,2023,GBP,USD,0.76,NMIP23005,Ameren Spring 2023,...,1660.2352,0.0000,0.0000,485.8376,60794.6800,219318.0640,126028.5200,0.0000,0.00,0.00
